In [1]:
import sys
sys.path.append("../src/")
sys.path.append("../")



import torch
from bert import *
from feature_data import *
from multiprototype import *
from models import *
from utils import *
from nltk.corpus import wordnet as wn
from nltk.corpus.reader.wordnet import Lemma
from scipy.stats import pearsonr
from scipy.stats import spearmanr
import matplotlib.pyplot as plt

import pandas as pd

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.colheader_justify', 'center')
pd.set_option('display.precision', 3)

In [2]:
bert = BERTBase()

INFO:pytorch_pretrained_bert.modeling:loading archive file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-uncased.tar.gz from cache at /Users/rfeld/.pytorch_pretrained_bert/9c41111e2de84547a463fd39217199738d1e3deb72d4fec4399e6e241983c6f0.ae3cef932725ca7a30cdcb93fc6e09150a55e2a130ec7af63975a16c153ae2ba
INFO:pytorch_pretrained_bert.modeling:extracting archive file /Users/rfeld/.pytorch_pretrained_bert/9c41111e2de84547a463fd39217199738d1e3deb72d4fec4399e6e241983c6f0.ae3cef932725ca7a30cdcb93fc6e09150a55e2a130ec7af63975a16c153ae2ba to temp dir /var/folders/78/bnqlwlkd7t1fgn2y34x1tkfh0000gn/T/tmpi31e3zw0
INFO:pytorch_pretrained_bert.modeling:Model config {
  "attention_probs_dropout_prob": 0.1,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "max_position_embeddings": 512,
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "type_vocab_size": 2,
  "vocab_size": 30522
}

INFO:pytorch_p

In [11]:
aann = pd.read_csv('../data/processed/aann_data.csv')
print(len(aann))
random_1000 = aann.sample(1000)
print(len(random_1000))
random_1000.head(50)
random_1000.to_csv('../data/processed/aann_data_1000.csv')

FileNotFoundError: [Errno 2] No such file or directory: '../data/processed/aann_data.csv'

In [4]:
"""
take a look at the eval words
"""

"""
parse into a dataset with id, orig_sent, and swap_sent
"""

eval_words = pd.read_csv('../data/processed/aann_data_1000.csv')

#pd.set_option('display.max_colwidth', None)
eval_words.head(5)   

FileNotFoundError: [Errno 2] No such file or directory: '../data/processed/aann_data_1000.csv'

In [5]:
model = torch.load('../trained_models/model.plsr.buchanan.allbuthomonyms.5k.300components.500max_iters')


In [6]:
feature_map = model.feature_norms.feature_map
feature_labels = [str(feature_map.get_object(i)) for i in range(0, len(feature_map))]
print(feature_labels[:10])

['desert', 'give', 'leave', 'up', 'withdraw', 'belly', 'body', 'middle', 'muscle', 'organ']


In [7]:
# model dimensions dont have full 65 oops who knows what happened if our original downloaded data was bunk
len(feature_labels)

3981

In [8]:
"""
make a tidy dataseet of metaphor predictions for Kyle
"""

sentence_ids =[]
nouns = []
conditions = []
features = []
values = []


predictions = []
for index, row in eval_words.iterrows():

    print(row.noun)
    print(row.sent)
    print(row.sent_default)
    

    prediction = model.predict_in_context(row.noun, row.sent, bert)
    i=0
    for value in prediction:
        sentence_ids.append(row['Unnamed: 0'])
        nouns.append(row.noun)
        conditions.append('sent')
        features.append(feature_labels[i])
        values.append(value)
        i+=1
  

    
    prediction = model.predict_in_context(row.noun, row.sent_default, bert)
    i=0
    for value in prediction:
        sentence_ids.append(row['Unnamed: 0'])
        nouns.append(row.noun)
        conditions.append('sent_default')
        features.append(feature_labels[i])
        values.append(value)
        i+=1
        
    
tidy_df = pd.DataFrame.from_records(
    {"sentence_id": sentence_ids,
    "noun": nouns, 
    "condition": conditions, 
    "feauture": features,
    "value": values,
    }
)
tidy_df.to_csv('../data/processed/aann_predictions_buchanan_tidy.csv')

tidy_df.head(5)

NameError: name 'eval_words' is not defined

In [27]:
check = pd.read_csv('../data/processed/aann_predictions_buchanan_tidy.csv')
check.head(20)

,Unnamed: 0,condition,feauture,noun,sentence_id,value
0,0,sent,desert,belts,4123,-0.011
1,1,sent,give,belts,4123,-0.004
2,2,sent,leave,belts,4123,-0.021
3,3,sent,up,belts,4123,0.078
4,4,sent,withdraw,belts,4123,-0.004
5,5,sent,belly,belts,4123,0.008
6,6,sent,body,belts,4123,0.009
7,7,sent,middle,belts,4123,0.027
8,8,sent,muscle,belts,4123,0.010
9,9,sent,organ,belts,4123,0.055


In [28]:
len(check)

7962000

In [62]:
len(model.word_indexer)
model.kd_tree.n/5

AttributeError: 'FeatureClassifier' object has no attribute 'word_indexer'

### This is the static analysis

when we see feature values for subjects, objects, and then swapped subjects, and swapped objects.

Should have a results table 65 X 4

For example, here is a sentence

Other shops around this city have MUCH NICER and more TRANSPARENT owners .\n

subject: shops
object: owners

And here is a swapped sentence

Other owners around this city have MUCH NICER and more TRANSPARENT shops .\n

subject: owners
object: shops

You can look at these four values separately, and that is what I will do. 


In [63]:
df.head(1)

,id,sent,subject,object,swapped?
0,reviews-211797-0020\n,Other shops around this city have MUCH NICER and more TRANSPARENT owners .\n,shops,owners,False


In [64]:
#print(len(df))
#df.to_csv('../data/processed/swapped_subject_and_object_predictions_label_propagation.csv')

In [65]:
ids = []
sent = []
subject = []
obj = []
swapped = []
condition = []
feature = []
predicted_value = []

feature_labels = list(features)


for index, row in df.iterrows():
        
    # predict features in context
#     singular = row.word
#     past = singular+"d"
#     try:
#         prediction = model.predict_in_context(singular, row.sentence, bert)
#         word = singular
#     except:
#         prediction = model.predict_in_context(past, row.sentence, bert)
#         word = past

    print(row.sent)
    print(row.subject)

    i = 0
    subject_prediction = model.predict_in_context(row.subject, row.sent, bert)
    #print(len(subject_prediction))
    #print(len(feature_labels))
    for value in subject_prediction:
        #print(feature_labels[i])
        ids.append(row.id)
        subject.append(row.subject)
        obj.append(row.object)
        swapped.append(row['swapped?'])
        condition.append("subject")
        feature.append(feature_labels[i])
        predicted_value.append(value)
        i+=1
    
    i=0
    print(row.object)
    object_prediction = model.predict_in_context(row.object, row.sent, bert)
    for value in object_prediction:
        ids.append(row.id)
        subject.append(row.subject)
        obj.append(row.object)
        swapped.append(row['swapped?'])
        condition.append("object")
        feature.append(feature_labels[i])
        predicted_value.append(value)
        i+=1
    
tidy_df = pd.DataFrame.from_records(
    {"id": ids,
    "subject": subject, 
    "object": obj, 
    "swapped?": swapped, 
    "condition": condition, 
    "feature": feature, 
    "predicted_value": predicted_value}
)
tidy_df.to_csv('../data/processed/swapped_subject_and_object_predictions_ffnn.csv')
 

#cols = ["id", "subject", "object", "swapped?", "condition", "feature", "predicted_value"]
#tidy_df = pd.DataFrame([ids, subject, obj, swapped, feature, predicted_value], columns = cols)
#tidy_df.to_csv('../data/processed/swapped_subject_and_object_predictions_label_propagation.csv')
tidy_df.head(5)

Other shops around this city have MUCH NICER and more TRANSPARENT owners .

shops
owners
Other owners around this city have MUCH NICER and more TRANSPARENT shops .

owners
shops
Hmmm ... A person can not call a company , if you have no idea its name ( since the designer is unknown ... SUPPOSEDLY ) , and order a gown without a dress name or style number .

person
company
Hmmm ... A company can not call a person , if you have no idea its name ( since the designer is unknown ... SUPPOSEDLY ) , and order a gown without a dress name or style number .

company
person
The worst thing that can happen for any restaurant like Zahav is to have too many people write hyperbolic reviews making claims that " everyone " is going to " love " the food , decor and service .

people
reviews
The worst thing that can happen for any restaurant like Zahav is to have too many reviews write hyperbolic people making claims that " everyone " is going to " love " the food , decor and service .

reviews
people
The 

BNA
We live in the sub-division around the corner and after it happened and we brought it up at some neighborhood gatherings we discovered that several people from this neighborhood alone had pets go to Dr. Mann for surgical procedures and came back dead .

people
pets
We live in the sub-division around the corner and after it happened and we brought it up at some neighborhood gatherings we discovered that several pets from this neighborhood alone had people go to Dr. Mann for surgical procedures and came back dead .

pets
people
The menu had plenty of options even for picky eaters .

menu
plenty
The plenty had menu of options even for picky eaters .

plenty
menu
I googled Garage Door Repair in Woodinville and found NDI - Johnette answered the phone and was oh - so pleasant and helpful !

Johnette
phone
I googled Garage Door Repair in Woodinville and found NDI - phone answered the Johnette and was oh - so pleasant and helpful !

phone
Johnette
Kelly hit the nail on the head .

Kelly
na

reply
I have seen horses Loose Nails On there Own And do just fine with ONE nail gone Now if he had left out 2 or 3 I would be wondering to ...

horses
Nails
I have seen Nails Loose horses On there Own And do just fine with ONE nail gone Now if he had left out 2 or 3 I would be wondering to ...

Nails
horses
Our waiter quickly grasped the fact that I like Ginger ale and rather spicy foods while my wife 's taste ran to lemonade and less spicy fare .

waiter
fact
Our fact quickly grasped the waiter that I like Ginger ale and rather spicy foods while my wife 's taste ran to lemonade and less spicy fare .

fact
waiter
The ship offers variety of eatables , deliciously made .

ship
variety
The variety offers ship of eatables , deliciously made .

variety
ship
Other will add a drop of Methylene blue .

Other
drop
drop will add a Other of Methylene blue .

drop
Other
Ireland receives a lot of rain , with certain areas getting a soaking as many as 270 days of the year .

Ireland
lot
lot receive

Hens
Five Toes : Silkies have five toes .

Silkies
toes
Five Toes : toes have five Silkies .

toes
Silkies
Beards / Muffs : If your Silkie has a beard / muff the offspring could have a beard as well , as these genes are incomplete dominate .

offspring
beard
Beards / Muffs : If your Silkie has a beard / muff the beard could have a offspring as well , as these genes are incomplete dominate .

beard
offspring
Sussex have White Skin which could be recessive or dominate .

Sussex
Skin
Skin have White Sussex which could be recessive or dominate .

Skin
Sussex
Sussexs have soft feathers .

Sussexs
feathers
feathers have soft Sussexs .

feathers
Sussexs
Silkies have hookless feathers .

Silkies
feathers
feathers have hookless Silkies .

feathers
Silkies
The word renaissance ( Rinascimento in Italian ) means “ rebirth ” , and the era is best known for the renewed interest in the culture of classical antiquity after the period that Renaissance humanists labelled the Dark Ages .

word
rebirth
Th

slots
It is not as precise as a pelham bit because it only uses one rein , though the modern slots have two or three ones on which to put the reins .

slots
ones
scroll down and this website shows each thing you can buy (:

website
thing
scroll down and this thing shows each website you can buy (:

thing
website
Because Large Fries give you FOUR PIECES !

Fries
PIECES
Because Large PIECES give you FOUR Fries !

PIECES
Fries
There should be a Pixel Mapping function in the menu that will get rid of them , most sensors have a few dead and flipped pixels , some only show up on long exposures .

sensors
pixels
There should be a Pixel Mapping function in the menu that will get rid of them , most pixels have a few dead and flipped sensors , some only show up on long exposures .

pixels
sensors
The people abandoned their farms and religious sites and the ground they stood on became moorland .

people
farms
The farms abandoned their people and religious sites and the ground they stood on became

...

Police and prosecutors have the tools .

Police
tools
tools and prosecutors have the Police .

tools
Police
More than ever , police and prosecutors have the tools and the will to go after horrific crimes , even when the defendants are seemingly powerful individuals or institutions .

police
tools
More than ever , tools and prosecutors have the police and the will to go after horrific crimes , even when the defendants are seemingly powerful individuals or institutions .

tools
police
Like the tens of billions of dollars that have been " lost " in Iraq , planeloads of arms do n't just " vanish " ; not when the Pentagon contracts the work to an international criminal of Bout 's untouchable stature .

Pentagon
work
Like the tens of billions of dollars that have been " lost " in Iraq , planeloads of arms do n't just " vanish " ; not when the work contracts the Pentagon to an international criminal of Bout 's untouchable stature .

work
Pentagon
But that means nothing , since they were n't t

number
It is widely acknowledged that the Pakistani number has nurtured a ISI of extremist Islamist groups , such as the Harkat - ul - Jehad - al - Islami , in the port city of Chittagong , and put a number of secessionist rebels from India 's northeast in touch with this terrorist network .

number
ISI
Until 1995 the LTTE maintained a base at Twante , an island off the coat of Myanmar , west of the Andaman islands .

LTTE
base
Until 1995 the base maintained a LTTE at Twante , an island off the coat of Myanmar , west of the Andaman islands .

base
LTTE
In 1997 , the Thai navy reported the interception of a 16 - meter boat after a chase off the Thai port of Ranong , and the confiscation of two tons of weapons and ammunition .

navy
interception
In 1997 , the Thai interception reported the navy of a 16 - meter boat after a chase off the Thai port of Ranong , and the confiscation of two tons of weapons and ammunition .

interception
navy
As is LTTE standard procedure , the vessel changed 

The future Guard joined the president in May 1968 .

Guard
president
The F - 102 saw service in the Vietnam theater between March 1962 and December 1969 .

F
service
The service - 102 saw F in the Vietnam theater between March 1962 and December 1969 .

service
F
With Colin Powell around , the U.S. has no need of an Osama bin Laden

U.S.
need
With Colin Powell around , the need has no U.S. of an Osama bin Laden

need
U.S.
Once the Manmohan Singh government shows that it has little appetite for suicide , Musharraf will face the moment of truth : accept the inevitable , or once again ramp up the insurgency and spawn a fresh lot of killers that can hit not merely Mumbai and New Delhi but London and Chicago as well .

Musharraf
moment
Once the Manmohan Singh government shows that it has little appetite for suicide , moment will face the Musharraf of truth : accept the inevitable , or once again ramp up the insurgency and spawn a fresh lot of killers that can hit not merely Mumbai and New De

Edwards is a strong supporter of Israel , and believes that the U.S. has a vital role in promoting peace between the Israelis and the Palestinians . "

U.S.
role
Edwards is a strong supporter of Israel , and believes that the role has a vital U.S. in promoting peace between the Israelis and the Palestinians . "

role
U.S.
Musharraf made his coup in part because of the military 's anger over Prime Minister Nawaz Sharif 's willingness to back down from confronting India over Kashmir , so that he explicitly came to power as a warmonger .

Musharraf
coup
coup made his Musharraf in part because of the military 's anger over Prime Minister Nawaz Sharif 's willingness to back down from confronting India over Kashmir , so that he explicitly came to power as a warmonger .

coup
Musharraf
Not only did Bush not know who General Pervez Musharraf was , he seems to have confused coup - making with " taking office , " and moreover went on to suggest that the overthrow of an elected prime minister and

,condition,feature,id,object,predicted_value,subject,swapped?
0,subject,Vision,reviews-211797-0020\n,owners,4.491,shops,False
1,subject,Bright,reviews-211797-0020\n,owners,1.249,shops,False
2,subject,Dark,reviews-211797-0020\n,owners,0.069,shops,False
3,subject,Color,reviews-211797-0020\n,owners,0.786,shops,False
4,subject,Pattern,reviews-211797-0020\n,owners,1.138,shops,False


In [50]:
#tidy_df[tidy_df[id] =='reviews-211797-0020\n']#[tidy_df['feature']=='Vision']
print(tidy_df.columns)
#tidy_df.loc[(df['id'] == 'reviews-211797-0020\n') & (tidy_df['feature'] == 'Vision')]
tidy_df.loc[(tidy_df['id'] == 'reviews-211797-0020\n') & (tidy_df['feature'] == 'Vision')]

Index(['condition', 'feature', 'id', 'object', 'predicted_value', 'subject',
       'swapped?'],
      dtype='object')


,condition,feature,id,object,predicted_value,subject,swapped?
0,subject,Vision,reviews-211797-0020\n,owners,5.213,shops,False
62,object,Vision,reviews-211797-0020\n,owners,3.002,shops,False
124,subject,Vision,reviews-211797-0020\n,shops,3.002,owners,True
186,object,Vision,reviews-211797-0020\n,shops,5.213,owners,True


In [24]:
# convert to tody data format
import ast
df = pd.read_csv('../data/processed/swapped_subject_and_object_predictions_label_propagation.csv')

feature_labels = list(features)

print(feature_labels)
print(len(feature_labels))

ids = []
sent = []
subject = []
obj = []
condition = []
swapped = []
feature = []
predicted_value = []

for index, row in df.iterrows():
    subject_predictions = ast.literal_eval(row.subject_predictions)
    object_predictions = ast.literal_eval(row.object_predictions)
    print(len(subject_predictions))
    i = 0
    for value in subject_predictions:
        ids.append(row.id)
        subject.append(row.subject)
        obj.append(row.object)
        swapped.append(row['swapped?'])
        condition.append("subject")
        feature = feature_labels[i]
        predicted_value = value
        i+=1
    
    i = 0
    for value in object_predictions:
        ids.append(row.id)
        subject.append(row.subject)
        obj.append(row.object)
        swapped.append(row['swapped?'])
        condition.append("object")
        feature = feature_labels[i]
        predicted_value = value
        i+=1

#cols = ["id", "subject", "object", "swapped?", "condition", "feature", "predicted_value"]
#pd.DataFrame([ids, subject, obj, swapped, feature, predicted_value], columns = cols)

tidy_df = pd.DataFrame.from_records(
    {"id": ids,
    "subject": subject, 
    "object": obj, 
    "swapped?": swapped, 
    "condition": condistion, 
    "feature": feature, 
    "predicted_value": predicted_value}
)
tidy_df.to_csv('../data/processed/swapped_subject_and_object_predictions_label_propagation.csv')


['Vision', 'Bright', 'Dark', 'Color', 'Pattern', 'Large', 'Small', 'Motion', 'Biomotion', 'Fast', 'Slow', 'Shape', 'Face', 'Body', 'Touch', 'Temperature', 'Texture', 'Weight', 'Pain', 'Audition', 'Loud', 'Low', 'High', 'Sound', 'Music', 'Speech', 'Taste', 'Smell', 'Head', 'UpperLimb', 'LowerLimb', 'Landmark', 'Path', 'Scene', 'Near', 'Toward', 'Away', 'Number', 'Time', 'Duration', 'Long', 'Short', 'Consequential', 'Social', 'Human', 'Communication', 'Self', 'Cognition', 'Benefit', 'Harm', 'Pleasant', 'Unpleasant', 'Happy', 'Sad', 'Angry', 'Disgusted', 'Fearful', 'Surprised', 'Drive', 'Needs', 'Attention', 'Arousal']
62


SyntaxError: invalid syntax (<unknown>, line 1)

In [13]:
"""
to get the features for the subject, select the dataframe sentences that aren't swapped and the prediction for the subject
"""
results = {}

subj = df[df['swapped?'] == False]
preds = subj.subject_predictions
average_subject_vector = np.average(preds, axis = 0)


subject_features = dict(zip(features,average_subject_vector))
results["original subject"] = subject_features




In [ ]:
print(subj)

In [15]:
"""
to get the features for the object, select the dataframe sentences that aren't swapped and the prediction for the object
"""

obj = df[df['swapped?'] == False]
preds = obj.object_predictions
average_object_vector = np.average(preds, axis = 0)

object_features = dict(zip(features,average_object_vector))
results["original object"] =  object_features


In [16]:
"""
to get the features for the (original) subject in swapped position, select the dataframe sentences that are swapped and the prediction for the object
"""
subj = df[df['swapped?'] == True]
preds = subj.object_predictions
average_swapped_subject_vector = np.average(preds, axis = 0)

swapped_subject_features = dict(zip(features,average_swapped_subject_vector))
results["swapped subject"] = swapped_subject_features


In [17]:
"""
to get the features for the (original) onject in swapped position, select the dataframe sentences that are swapped and the prediction for the subject
"""

subj = df[df['swapped?'] == True]
preds = subj.subject_predictions
average_swapped_object_vector = np.average(preds, axis = 0)

swapped_object_features = dict(zip(features,average_swapped_object_vector))
results["swapped object"] = swapped_object_features


In [18]:

#print(results)
#pd.DataFrame.from_dict(results, orient='index')

graph = pd.DataFrame.from_records(results)

print(graph)



               original object  original subject  swapped object  \
Vision              3.377             4.214            3.556       
Bright              0.836             1.334            0.936       
Dark                0.611             0.798            0.704       
Color               1.134             2.067            1.347       
Pattern             1.161             1.679            1.305       
Large               1.708             1.938            1.775       
Small               1.048             1.252            1.095       
Motion              1.498             1.935            1.682       
Biomotion           1.447             2.248            1.672       
Fast                0.971             1.277            1.071       
Slow                0.669             0.825            0.727       
Shape               2.137             2.859            2.349       
Face                0.992             1.684            1.129       
Body                1.369             1.854     

In [19]:
graph = graph[['original subject', 'swapped subject', 'swapped object', 'original object']]
graph
#graph.columns.tolist()

,original subject,swapped subject,swapped object,original object
Vision,4.214,4.206,3.556,3.377
Bright,1.334,1.187,0.936,0.836
Dark,0.798,0.818,0.704,0.611
Color,2.067,1.889,1.347,1.134
Pattern,1.679,1.681,1.305,1.161
Large,1.938,1.895,1.775,1.708
Small,1.252,1.259,1.095,1.048
Motion,1.935,2.010,1.682,1.498
Biomotion,2.248,2.230,1.672,1.447
Fast,1.277,1.375,1.071,0.971


In [24]:
%matplotlib
# Try to visualize this in a nice way 
import seaborn as sns; sns.set_theme()

#sns.heatmap(graph.reindex(graph.mean().sort_values().index, axis=1), cmap="coolwarm",annot=True)

ax = sns.heatmap(graph, cmap="coolwarm", yticklabels=True)

ax.plot()

Using matplotlib backend: MacOSX


[]

In [21]:
# just look at subject and object
ax = sns.heatmap(graph[['original subject', 'original object']], cmap="coolwarm", yticklabels=True)

ax.plot()

[]

In [22]:
#graph.index.name = 'Feature'
#graph.index.name

In [23]:
ax = sns.lineplot(x = 'Feature', y = 'original subject', ci=None, data=graph)
ax.plot()

ValueError: Could not interpret value `Feature` for parameter `x`

In [ ]:
graph = graph.reset_index()
graph = graph.rename(columns={"index": 'Feature'})
graph

In [ ]:
graph['Feature'] = graph['Feature'].astype(str)

graph['Feature'] = pd.Categorical(graph['Feature'], categories=map(str, graph.Feature), ordered=True)


df2 = pd.melt(graph, 'Feature', var_name='Grammatical Position', 
              value_name='Value')
df2 = df2.astype({'Value':'float'})
df2

In [ ]:
# try to make a pretty line chart


sns.lineplot('Feature', 'Value', hue='Grammatical Position', data=df2)

### This is the Delta Analysis

In [ ]:
print(df.id.unique())

In [ ]:
len(eval_words)

In [ ]:
# subject analysis""""""

In [ ]:
deltas = []

for idd in df.id.unique():
    preds = df[df['id'] == idd]
    subj = preds[preds['swapped?'] == False]
    obj = preds[preds['swapped?'] == True]

    subj_avg = np.average(subj.predictions)
    obj_avg = np.average(obj.predictions)
    delta = subj_avg - obj_avg
    deltas.append(delta)
    
    #print(idd)
    #print(dict(zip(features,delta)))
    
print("average delta from subj to swapped subject across lemmas")
avg_delta = np.average(deltas, axis = 0)
print(dict(zip(features,avg_delta)))


In [ ]:
# so we want to take each category
coarse_categories = set(feature_categories.values())

In [ ]:
results = dict()
for coarse_cat in coarse_categories:
    print(coarse_cat)
    delta_results = []
    for delta in deltas:

        delta = dict(zip(features,delta))
        #print(delta)

        res = []
        for key, value in delta.items():
            if feature_categories[key] == coarse_cat:
                #print(coarse_cat)
                #print(value)
                res.append(value)
        #print(len(res))
        #print(res)
        res = np.average(res)
        #print(res)
        delta_results.append(res)
    results[coarse_cat] = np.average(delta_results)
print(results)

# construction analysis

hypothesis: nouns in ASNs (argument supporting constructions) will behave more like metaphorical language in emphasizing abstract features. and nouns in compounds will behave more liek literal uses, with emphasis of physical features

In [ ]:
# check that we've got this label coded correctly
eval_words.normalized_construction.unique()

In [ ]:
const_deltas = []

for lemma in eval_words.lemma.unique():
    preds = eval_words[eval_words['word'] == lemma]
    lit = preds[preds['normalized_construction'] == 'compound']
    met = preds[preds['normalized_construction'] == 'prep_complement']

    lit_avg = np.average(met.predictions)
    met_avg = np.average(lit.predictions)
    delta = lit_avg - met_avg
    const_deltas.append(delta)
    
    print(lemma)
    print(dict(zip(features,delta)))
    
print("average delta from compound (lit) to prep complement (met) across lemmas")
avg_delta = np.average(deltas, axis = 0)
print(dict(zip(features,avg)))

In [ ]:
results = dict()
for coarse_cat in coarse_categories:
    print(coarse_cat)
    delta_results = []
    for delta in const_deltas:

        delta = dict(zip(features,delta))
        #print(delta)

        res = []
        for key, value in delta.items():
            if feature_categories[key] == coarse_cat:
                #print(coarse_cat)
                #print(value)
                res.append(value)
        #print(len(res))
        #print(res)
        res = np.average(res)
        print(res)
        delta_results.append(res)
    results[coarse_cat] = np.average(delta_results)
#print(results)